In [234]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Part B: Predictive Modelling
## I. Feature engineering

In [235]:
df = pd.read_csv('data/zomato_df_final_data.csv')
df.isna().sum()

address             0
cost              346
cuisine             0
lat               192
link                0
lng               192
phone               0
rating_number    3316
rating_text      3316
subzone             0
title               0
type               48
votes            3316
groupon             0
color               0
cost_2            346
cuisine_color       0
dtype: int64

In [236]:
df_rating_text = pd.get_dummies(df['rating_text'])
df = df.drop(columns = ["address", "link", "phone", "title", "color", "cuisine_color", "type", "cuisine", "rating_text", "lat", "lng"], axis = 1)
df.head(3)

,cost,rating_number,subzone,votes,groupon,cost_2
0,50.0,4.0,CBD,1311.0,False,5.243902
1,80.0,4.6,"The Grounds of Alexandria, Alexandria",3236.0,False,7.560976
2,120.0,4.9,"The Star, Pyrmont",1227.0,False,10.650407


In [237]:
df_rating_text.head(3)

,Average,Excellent,Good,Poor,Very Good
0,False,False,False,False,True
1,False,True,False,False,False
2,False,True,False,False,False


In [238]:
df_rating_text['bin_rating'] = 0
# if in df_rating_text['Poor'] == True or df_rating_text['Average'] == True then put 1 in df_rating_text['bin_rating'] else put 2
df_rating_text['bin_rating'] = df_rating_text.apply(lambda x: 1 if x['Poor'] == 1 or x['Average'] == 1 else 2, axis=1)
df_rating_text.head(10)

,Average,Excellent,Good,Poor,Very Good,bin_rating
0,False,False,False,False,True,2
1,False,True,False,False,False,2
2,False,True,False,False,False,2
3,False,True,False,False,False,2
4,False,True,False,False,False,2
5,False,True,False,False,False,2
6,False,True,False,False,False,2
7,False,False,False,False,True,2
8,False,False,False,False,True,2
9,False,True,False,False,False,2


In [239]:
df.groupon = df.groupon.astype(int)

# cleaning up the subzone column
df['subzone'] = df['subzone'].apply(lambda x: x.split(',')[-1].strip() if ',' in x else x)

# encoding the subzone column
label_encoder = LabelEncoder()
df['subzone_encoded'] = label_encoder.fit_transform(df['subzone'])
df.head(3)

,cost,rating_number,subzone,votes,groupon,cost_2,subzone_encoded
0,50.0,4.0,CBD,1311.0,0,5.243902,48
1,80.0,4.6,Alexandria,3236.0,0,7.560976,1
2,120.0,4.9,Pyrmont,1227.0,0,10.650407,245


In [240]:
df.isna().sum()

cost                346
rating_number      3316
subzone               0
votes              3316
groupon               0
cost_2              346
subzone_encoded       0
dtype: int64

In [241]:
# handling the missing values
# fill the rating column with the mean
df['rating_number'] = df.groupby('subzone_encoded')['rating_number'].transform(lambda x: x.fillna(x.mean()))
df['rating_number'] = df['rating_number'].fillna(df['rating_number'].mean())
# fill the votes column with the mean
df['votes'] = df.groupby('subzone_encoded')['votes'].transform(lambda x: x.fillna(x.mean()))
df['votes'] = df['votes'].fillna(df['votes'].mean())
df.isna().sum()

cost               346
rating_number        0
subzone              0
votes                0
groupon              0
cost_2             346
subzone_encoded      0
dtype: int64

In [242]:
df['cost'] = df.groupby('subzone_encoded')['cost'].transform(lambda x: x.fillna(x.mean()))
df['cost'] = df['cost'].fillna(df['cost'].mean())

df['cost_2'] = df.groupby('subzone_encoded')['cost_2'].transform(lambda x: x.fillna(x.mean()))
df['cost_2'] = df['cost_2'].fillna(df['cost_2'].mean())

df = df.drop(columns = ["subzone"], axis = 1)
df.isna().sum()

cost               0
rating_number      0
votes              0
groupon            0
cost_2             0
subzone_encoded    0
dtype: int64

## II. Regression Modelling

In [243]:
X = df.drop(columns=['rating_number'])
y = df['rating_number']

#### Build a linear regression model (model_regression_1) to predict the restaurants rating (numeric rating) from other features (columns) in the dataset. 
Please consider splitting the data into train (80%) and test (20%) sets.

In [244]:
# Build a linear regression model (model_regression_1) to predict the restaurants rating (numeric rating) from other features (columns) in the dataset. Please consider splitting the data into train (80%) and test (20%) sets.
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
model_regression_1 = LinearRegression()
model_regression_1.fit(X_train, y_train)
# predictions
y_pred = model_regression_1.predict(X_test)
# print('Coefficients: \n', model_regression_1.coef_)
# Evaluate the model using the Mean Squared Error and R-squared metrics.
print('Regression Model 1:')
print('Mean squared error: %.2f' % mean_squared_error(y_test, y_pred))
print('Coefficient of determination: %.2f' % r2_score(y_test, y_pred))


Regression Model 1:
Mean squared error: 0.11
Coefficient of determination: 0.26


#### Build another linear regression model (model_regression_2) with using the Gradient Descent as the optimisation function

In [245]:
# Build another linear regression model (model_regression_2) with using the Gradient Descent as the optimisation function
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model_regression_2 = SGDRegressor(loss="squared_error", max_iter=150, random_state=0)
model_regression_2.fit(X_train_scaled, y_train)
# predictions
y_pred = model_regression_2.predict(X_test_scaled)
# print('Coefficients: \n', model_regression_2.coef_)
# Evaluate the model using the Mean Squared Error and R-squared metrics.
print('Regression Model 2:')
print('Mean squared error: %.2f' % mean_squared_error(y_test, y_pred))
print('Coefficient of determination: %.2f' % r2_score(y_test, y_pred))

Regression Model 2:
Mean squared error: 0.11
Coefficient of determination: 0.24


In [246]:
df.dtypes

cost               float64
rating_number      float64
votes              float64
groupon              int32
cost_2             float64
subzone_encoded      int32
dtype: object

## III. Classification Modelling

In [247]:
df_rating_text.head(3)

,Average,Excellent,Good,Poor,Very Good,bin_rating
0,False,False,False,False,True,2
1,False,True,False,False,False,2
2,False,True,False,False,False,2


In [248]:
df['bin_rating'] = df_rating_text['bin_rating']
df.head(3)

,cost,rating_number,votes,groupon,cost_2,subzone_encoded,bin_rating
0,50.0,4.0,1311.0,0,5.243902,48,2
1,80.0,4.6,3236.0,0,7.560976,1,2
2,120.0,4.9,1227.0,0,10.650407,245,2


In [249]:
# build a logistic regression model (model_classification_3) to predict the restaurants rating (binary rating) from other features (columns) in the dataset. Please consider splitting the data into train (80%) and test (20%) sets.
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

X = df.drop(columns=['bin_rating'])
y = df['bin_rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
model_classification_3 = LogisticRegression()
model_classification_3.fit(X_train, y_train)
# predictions
y_pred = model_classification_3.predict(X_test)
# print('Coefficients: \n', model_classification_3.coef_)
# Evaluate the model using the Accuracy, Confusion Matrix and Classification Report metrics.
print('Classification Model 3:')
print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))
print('Confusion Matrix: \n', confusion_matrix(y_test, y_pred))
print('Classification Report: \n', classification_report(y_test, y_pred))

Classification Model 3:
Accuracy: 0.77
Confusion Matrix: 
 [[728 256]
 [223 893]]
Classification Report: 
               precision    recall  f1-score   support

           1       0.77      0.74      0.75       984
           2       0.78      0.80      0.79      1116

    accuracy                           0.77      2100
   macro avg       0.77      0.77      0.77      2100
weighted avg       0.77      0.77      0.77      2100
